This project will utilize 2 different machine learning classifiers to try and reach the highest possiable accuarcy in predicting whether someone has parkinsons according to the UCI ML Parkinsons dataset. This Data set uses the status column to indicate whether someone has Parkinsons or not. A value of 1 means the persons has Parkinosons, and a value of 0 means a person does not have Parkinosons.

In [ ]:
import numpy as np
import pandas as pd 
import os, sys
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
ParkinsonsData  = pd.read_csv("../input/parkinsons/parkinsons.txt")
ParkinsonsData= ParkinsonsData.drop('name',1)
ParkinsonsData

The name collumn is dropped as it is not neccesacry. Idintifiaction of patients will not aid our models ability to learn later in the proccess, all we need is test results. 

In [ ]:
ParkinsonsData.shape

Using pandas shape property we can see that the file has a total of 195 columns, out of curioustiy I'll check to see the number of entries with parkinsons and the number of entries without parkinsons.

In [ ]:
hasParkinsons = ParkinsonsData.loc[ParkinsonsData.status == 1]
DoesNotHavePD = ParkinsonsData.loc[ParkinsonsData.status == 0]

In [ ]:
hasParkinsons.shape

In [ ]:
DoesNotHavePD.shape

As we can see, this data set is composed of 147 entries of people with parkinsons and 48 people without parkinsons.

In [ ]:
X = ParkinsonsData.loc[:, ParkinsonsData.columns != 'status']
y = (ParkinsonsData.status).values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 0)

Do to the fact that the data set is only 195 entries, I will leave out 25% of the data for validation to ensure the model sees enough entries in the training process.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Define the models
model_1 = RandomForestClassifier(n_estimators=100, random_state=0)


In [ ]:
from sklearn.metrics import accuracy_score

model_1.fit(X_train, y_train)
preds = model_1.predict(X_val)

accuracy = accuracy_score(y_val, preds)
print("Model Accuracy:")
print(accuracy * 100)

Now we will use Keras Sequential Classifeir to build a model and compare the accuracy score. 

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, callbacks
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)
model_2 = keras.Sequential([

    
    layers.Dense(32, activation= 'relu', kernel_initializer = 'random_normal', input_dim = 22),
    layers.Dropout(0.3),
    layers.BatchNormalization(),

    layers.Dense(16, activation= 'relu', kernel_initializer = 'random_normal'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    
    layers.Dense(8, activation= 'relu', kernel_initializer = 'random_normal'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),

    layers.Dense(1, activation= 'sigmoid', kernel_initializer = 'random_normal'),   
])

model_2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

history = model_2.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=10,
    epochs=1000,
    callbacks=[early_stopping], # put your callbacks in a list
    verbose=0,  # turn off training log
)

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))


In [ ]:
tscores = model_2.evaluate(X_val, y_val)
print("Test Accuracy: %.2f%%" %(tscores[1]*100))